# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
display(data.head(15))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Имеем таблицу с 12-ю правильно названными колонками и 21525 строками. Имеются пропуски в колонках days_employed, total_income. Тип данных целые, вещественные числа и строки. В колонке days_employed отрицательные значения, следует разобраться. В столбце education следует привести написание к единообразию, например к прописным буквам. Так же следует разбить на категории столбец purpose.

Описание данных:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print('Всего пропущено строк:', len(data[data['days_employed'].isna()])) # Проверяем количество пропусков в колонке "Трудовой стаж"
#display(data[data['total_income'].isna()]) # посмотрим глазами на явные несоответствия.

#print(data.sort_values(by='children')) 
#print(data['children'].value_counts())

#print(data.sort_values(by='days_employed'))

#print(data.sort_values(by='dob_years'))
#print(data['dob_years'].value_counts())

days_employed_mean = data['days_employed'].mean()
print('Среднее количество трудовых дней:', days_employed_mean) 

total_income_median = data['total_income'].median() # медиана по заработку
print('Средний ежемесячный доход:', total_income_median)

dob_years_mean = int(data['dob_years'].mean()) # средний возраст
print('Средний возраст:', dob_years_mean)

data['days_employed'] = data['days_employed'].fillna(days_employed_mean)
data['total_income'] = data['total_income'].fillna(total_income_median)
data['dob_years'] = data['dob_years'].replace(0,dob_years_mean)
data['children'] = data['children'].replace(20, 2).abs()

#data.info()



Всего пропущено строк: 2174
Среднее количество трудовых дней: 63046.49766147338
Средний ежемесячный доход: 145017.93753253992
Средний возраст: 43


In [3]:
print(data['days_employed'].max() / 365 / 24) # проверили максимальные значения стажа.
print(data['days_employed'].min() / 365) # и минимальные значения

#data.sort_values(by='days_employed').head(15)

45.8624886387363
-50.38068465909146


**Вывод**

Обнаружены пропуски в колонках "трудовой стаж" и "ежемесячный доход".
Пропуски скорее всего вызваны техническими причинами, т.к. при сборе данных о клиенте, перед принятием решения о выдаче кредита, кредитный менеджер обязательно собирает информацию о доходе. В пользу гипотезы о техническом сбое говорит и то, что в этих колонках в одних и тех же строках пропущены значения.
Так как для решения этого кейса конкретные значения трудового стажа не важны, то принято решение заполнить пропуски по следующему принципу:
- "Трудовой стаж" - по среднему арифметическому.
- "Средний ежемесячный доход" - медианное значение, чтобы исключить аномально большие или маленькие значения.

При проверке колонки "Дети" обнаружены аномальные значения, отрицательные и слишком большие, 20 детей. Процент некорректных значений не большой (и это топ-3 популярного количества детей), можно было бы удалить эти строки, но мы отрицательные значения уберем методом .abs(), а значение "20" превратим в "2", т.к.логично предположить, что тут операторская ошибка при наборе данных.

При проверке колонки "Возраст" обнаружен пропуск возраста, обозначенный "0", замена средним арифметическим.

### Замена типа данных

In [4]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

#data.info()

**Вывод**

Для удобства заменим вещественные числа на целые. Для этого применим метод .astype(int) к колонкам трудового стажа и среднего дохода.

### Обработка дубликатов

In [5]:
print('Дупликатов в таблице:', data.duplicated().sum()) 
#print('Уникальные значения в колонке \"Образование\":\n', data['education'].value_counts())

Дупликатов в таблице: 54


In [6]:
data['education'] = data['education'].str.lower() #приведем написание к единому, прописному

print(data['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [7]:
#print(data['family_status'].value_counts())

In [8]:
#print(data['income_type'].value_counts())

In [9]:
#print(data['purpose'].value_counts())

In [10]:
print('Дупликатов в таблице:', data.duplicated().sum()) # проверим наличие явных дубликатов после обработки и удалим их
data = data.drop_duplicates().reset_index(drop=True)
print('Дупликатов в таблице:', data.duplicated().sum())

Дупликатов в таблице: 71
Дупликатов в таблице: 0


In [11]:
data.info()
#display(data.head(15))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21454 non-null  int64 
 1   days_employed     21454 non-null  int64 
 2   dob_years         21454 non-null  int64 
 3   education         21454 non-null  object
 4   education_id      21454 non-null  int64 
 5   family_status     21454 non-null  object
 6   family_status_id  21454 non-null  int64 
 7   gender            21454 non-null  object
 8   income_type       21454 non-null  object
 9   debt              21454 non-null  int64 
 10  total_income      21454 non-null  int64 
 11  purpose           21454 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Появление дубликатов есть следствие отсутствия стандартизированных форм для заполнения ячеек.
В таблице были удалены явные дубликаты и приведены к единообразию значения во всех колонках, кроме колонки "цель получения кредита". К ней применим лемматизацию.

### Лемматизация

In [12]:
from pymystem3 import Mystem
m = Mystem()

In [13]:
def lemma_for_id(row): # напишем функцию для создания столбца с категориями
    purpose_lemma = m.lemmatize(row['purpose'])
    if 'автомобиль' in purpose_lemma:
        return 'автомобиль'
    if ('жилье' in purpose_lemma) or ('недвижимость' in purpose_lemma):
        return 'недвижимость'
    if 'свадьба'  in purpose_lemma:
        return 'свадьба'
    if 'образование' in purpose_lemma:
        return 'образование'
    
data['purpose_id'] = data.apply(lemma_for_id, axis=1)

display(data.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


**Вывод**

Провели лемматизацию, выделили категории в отдельной колонке: свадьба, автомобиль, недвижимость, образование.

### Категоризация данных

Выделим категории в зависимости от уровня дохода.

Федеральная служба государственной статистики разделяет уровень жизни россиян в зависимости от доходов на следующие категории:

- крайняя нищета (доходы ниже прожиточного минимума — до 7-8 тыс.р.)
- нищета (доходы от одного до двух прожиточных минимума — от 8 до 12 тыс.р.)
- бедность (доходы от 12 до 20 тысяч рублей в месяц)
- выше бедности (доходы от 20 до 30 тысяч рублей в месяц)
- средний достаток — (доходы от 30 до 60 тысяч рублей в месяц)
- состоятельные — (доходы от 60 до 90 тысяч рублей в месяц)
- богатые — (доходы от 90 тысяч рублей в месяц)
- сверхбогатые — (доходы свыше 150 тысяч рублей в месяц)

Но нам такподробно не нужно и мы выделим свои категории:
- Низкий доход - до 29 999 руб.
- Средний доход - от 30 000 до 59 999 руб.
- Высокий доход - от 60 000 до 149 999 руб.
- Сверхвысокий доход - от 150 000 руб.

In [15]:
#def income_grouped(cash):
#    if data['total_income'].min() <= cash <= data['total_income'].median():
#        return 'низкий'
#    if data['total_income'].median() <= cash <= data['total_income'].max():
#        return 'высокий'
    
#print(income_grouped(150000))

def income_grouped(cash):
    if 0 <= cash <= 59999:
        return 'низкий доход'
    if 60000 <= cash <= 149999:
        return 'средний доход'
    if 150000 <= cash <= 500000:
        return 'высокий доход'
    return 'сверхбогатые'

data['total_income_id'] = data['total_income'].apply(income_grouped)

In [16]:
#print(data.sort_values('total_income', ascending=False).head(20))
#print(data['total_income_id'].value_counts())


**Вывод**

Категоризировал колонку с доходом: "высокий", "средний", "низкий".
Так же сделал категории для колонки "Цель".
В остальной таблице так или иначе уже есть категории.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [17]:
#children = data[(data['children_id'] == 'есть дети') & (data['debt'] == 0)]

children = data[['children','debt']]

#print(children.query('debt == 0').groupby('children').count())
#print(children.query('debt == 1').groupby('children').count())

print(children.query('debt == 0').groupby('children').count() / children.query('debt == 1').groupby('children').count())

               debt
children           
0         12.255880
1          9.910112
2          9.534653
3         11.222222
4          9.250000
5               NaN


**Вывод**

Из данных представленных выше мы видим, что с возрастанием количества детей уменьшается риск не возврата долга вовремя. Но семьи с тремя детьми выделяются на общем фоне. Это может быть связано со слишком маленькой выборкой по этой категории.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [18]:
family = data[['family_status','debt']]

#print(family.query('debt == 0').groupby('family_status').count())
#print(family.query('debt == 1').groupby('family_status').count())

print(family.query('debt == 0').groupby('family_status').count() / family.query('debt == 1').groupby('family_status').count())

                            debt
family_status                   
Не женат / не замужем   9.255474
в разводе              13.058824
вдовец / вдова         14.222222
гражданский брак        9.698454
женат / замужем        12.253491


**Вывод**

Из представленных выше данных видим соотношение количества должников среди разных категорий.

Чаще всего не отдают долг вовремя вдовцы и те кто в разводе, что в-принципе логично.

Самое меньшее количество должников находится среди необременённых официальными отношениями.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [19]:
income = data[['total_income_id', 'debt']]

#print(income.query('debt == 0').groupby('total_income_id').count())
#print(income.query('debt == 1').groupby('total_income_id').count())

print(income.query('debt == 0').groupby('total_income_id').count() / income.query('debt == 1').groupby('total_income_id').count())

                      debt
total_income_id           
высокий доход    11.587079
низкий доход     15.448980
сверхбогатые     14.857143
средний доход    10.867495


**Вывод**

Исходя из полученного результата видно, люди имеющие средний доход реже всего допускают просрочку по возврату долга. А бедняки и богачи напротив имеют самые высокие риски.

- Как разные цели кредита влияют на его возврат в срок?

In [20]:
purpose = data[['purpose_id', 'debt']]

#print(purpose.query('debt == 0').groupby('purpose_id').count())
#print(purpose.query('debt == 1').groupby('purpose_id').count())

print(purpose.query('debt == 0').groupby('purpose_id').count() / purpose.query('debt == 1').groupby('purpose_id').count())

                   debt
purpose_id             
автомобиль     9.684864
недвижимость  12.824808
образование    9.845946
свадьба       11.494624


In [21]:
data.groupby('purpose_id')['debt'].agg(['count', 'mean'])

,count,mean
purpose_id,,
автомобиль,4306,0.093590
недвижимость,10811,0.072334
образование,4013,0.092200
свадьба,2324,0.080034


In [22]:
debt_0 = data[data['debt'] == 0]
debt_1 = data[data['debt'] == 1]

In [23]:
print(debt_0['debt'].value_counts())
print(debt_1['debt'].value_counts())

0    19713
Name: debt, dtype: int64
1    1741
Name: debt, dtype: int64


**Вывод**

Просрочку по кредиту чаще всего допускают люди которые берут кредит на недвижимость.

Реже всего автомобилисты.

## Шаг 4. Общий вывод

В результате первичного ознакомления с входными данными была проведена работа по их предобработке, которая включила в себя: определение и заполнение пропущенных значений, замену типов данных, удаление дубликатов, выделение лемм и категоризацию данных.

При обработке данных были обнаружены артефакты, и некорректные значения. Например было исправлено отсутствие значение возраста "0" на средний возраст клиентов. Колонка трудовой стаж была оставлена без изменений, так как не влияет на решение поставленных вопросов. Хотя есть несколько гипотез касающихся отрицательных (или высоких\низких) значений в этой колонке. Например, некоторе значения могут быть занесены в часах вместо дней.

В целом, все важные, имеющие значение данные были исправлены для дальнейшей работы с таблицей.

Основываясь на проведенной работе можно сделать общий вывод, что в целом закономерности прослеживаются и то что понятно интуитивно, подтверждается анализом данных. 

Например, люди с детьми более ответственно относятся к долговым обязательствам. Или что люди разведенные или вдовцы имеют более высокий риск просрочки. А вот например тот факт, что богатые люди чаще допускают просрочку по кредиту, чем средний класс, не всегда очевидно.

Можно было провести более глубокий анализ, и выявить больше закономерностей, например отследить закономерности по возврату долга в зависимости от образования, возраста, статуса клиентов. И с большой долей вероятности этот дополнительный анализ подтвердил бы общий вывод проекта.

Общая рекомендация заказчику: 

Основываясь на проведённом анализе можно проследить определенные закономерности и учесть их при построении модели кредитного скоринга.